In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Zach Flowers, Murtaza Safdari <musafdar@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
import random
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
import os, sys
import multiprocessing
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebooks.notebook_helpers import *
from fnmatch import fnmatch
import scipy.stats as stats
from math import ceil
from numpy import savetxt

os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import parser_arguments
import importlib
importlib.reload(run_script)
# from i2c_gui.chips.etroc2_chip import register_decoding

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
# chip_names = ["ET2_EPIR_Pair4"]
# chip_fignames = ["ET2 EPIR Pair 4"]
chip_names = ["ET2p03_BARE1"]
chip_fignames = chip_names
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: /dev/ttyACM0'
port = "/dev/ttyACM0"
# I2C addresses for the pixel block and WS
# chip_addresses = [0x60, 0x61, 0x62]
chip_addresses = [0x60]
# ws_addresses = [None, None, None, None]
ws_addresses = [0x40]

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,ws_addresses,chip_names,[("1","1"),("1","1"),("1","1"), ("1","1")])

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
# (WS Prep Pixel and Peri) - 0 - (disable & auto_cal all pixels) - (disable default all pixels) - (auto_TH_CAL) - (set basic peripherals) - (peripheral reg check) -  (pixel ID check)
i2c_conn.config_chips('00100101')
# i2c_conn.config_chips('00001111') # disable and more fun things
# i2c_conn.config_chips('00000100') # set basic peripherals
# i2c_conn.config_chips('00010000') # calibrate only
# i2c_conn.config_chips('00000001') # pixel ID check (I2C check)
i2c_conn.config_chips('10000000') # ws init
# i2c_conn.disable_all_pixels(chip_address=chip_addresses[1])

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
plt.figure()
plt.show()

In [ ]:
histdir = Path('../ETROC-History/')
# histdir = Path('/media/daq/X9/DESY_Jun2024_AIDA/ETROC-History/')
histdir.mkdir(exist_ok=True)
histfile = histdir / 'BaselineHistory.sqlite'
i2c_conn.save_baselines(chip_fignames,fig_path,histdir,histfile)

# Define pixels of interest

In [ ]:
# WS Pixel
row_list = [0]
col_list = [14]

scan_list = list(zip(row_list, col_list))

# col_list, row_list = np.meshgrid(np.arange(16),np.arange(16))
# scan_list = list(zip(row_list.flatten(),col_list.flatten()))

print(scan_list)

In [ ]:
for chip_address,chip_name in zip(chip_addresses,chip_names):
    chip = i2c_conn.get_chip_i2c_connection(chip_address)
    row_indexer_handle,_,_ = chip.get_indexer("row")
    column_indexer_handle,_,_ = chip.get_indexer("column")

    chip.read_all_block("ETROC2", "Peripheral Config")
    handle = chip.get_decoded_display_var("ETROC2", "Peripheral Config", "chargeInjectionDelay") # User tunable delay of Qinj pulse
    handle.set(hex(0x0e))
    chip.write_all_block("ETROC2", "Peripheral Config")

    # for row, col in scan_list:
    #     print(f"Operating on chip {hex(chip_address)} Pixel ({row},{col})")
    #     column_indexer_handle.set(col)
    #     row_indexer_handle.set(row)
    #     # i2c_conn.pixel_decoded_register_write("QSel", format(0x1f, '05b'), chip)
    #     # i2c_conn.pixel_decoded_register_write("TH_offset", format(0x19, '06b'), chip)
    #     i2c_conn.pixel_decoded_register_write("QSel", format(0x1f, '05b'), chip)
    #     i2c_conn.pixel_decoded_register_write("TH_offset", format(0xf, '06b'), chip)
    #     i2c_conn.pixel_decoded_register_write("QInjEn", '1', chip)
    #     i2c_conn.pixel_decoded_register_write("RFSel", format(0x00, '02b'), chip=chip)

    del row_indexer_handle, column_indexer_handle, chip

## Run in terminal to collect waveforms:

In [ ]:
# python run_script.py -f --useIPC --hostname 192.168.2.3 -t 300 -o ws_testing_5 -v -w -s 0x0000 -p 0x0023 -d 0x1800 -a 0x0091 --start_DAQ_pulse --stop_DAQ_pulse --skip_binary --ws_testing --ws_i2c_port /dev/ttyACM0 --ws_chip_address 0x60 --ws_address 0x40 --start_dev_ws_beam_fc

# python run_script.py -f --useIPC --hostname 192.168.2.3 -t 300 -o ws_testing_qinj_nb -v -w -s 0x0000 -p 0x0023 -d 0x1800 -a 0x00a1 --start_DAQ_pulse --stop_DAQ_pulse --skip_binary --ws_testing --ws_i2c_port /dev/ttyACM0 --ws_chip_address 0x60 --ws_address 0x40 --start_dev_qinj_fc

In [ ]:
hostname = "192.168.2.3"
polarity = "0x0023" # "0x0023" or "0x0027"
out_dir = "ws_testing_qinj_nb"
trigger_bit_delay = "0x1800"
channel_enable = "0x0001"
ws_i2c_port = "/dev/ttyACM0"
ws_chip_address = "0x60"
ws_address = "0x40"

# --clear_fifo
# --check_valid_data_start
# -- start_dev_qinj_fc  stop_dev_qinj_fc
# -- start_dev_ws_beam_fc stop_dev_ws_beam_fc

In [ ]:
parser = parser_arguments.create_parser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname {hostname} -t 125 -o {out_dir} -v -w -s 0x0000 -p {polarity} -d {trigger_bit_delay} -a {channel_enable} --clear_fifo --start_DAQ_pulse --stop_DAQ_pulse --ws_testing --ws_i2c_port {ws_i2c_port} --ws_chip_address {ws_chip_address} --ws_address {ws_address} --compressed_binary".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, None))
process.start()

IPC_queue.put('memoFC Start Triggerbit QInj L1A BCR')
while not IPC_queue.empty():
    pass
time.sleep(120)
IPC_queue.put('stop DAQ')
IPC_queue.put('memoFC Triggerbit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

del IPC_queue, process, parser

# Calibrate FC for all I2C

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.asyResetGlobalReadout(chip_address, chip=None)
    i2c_conn.asyAlignFastcommand(chip_address, chip=None)

# Calibrate PLL

In [ ]:
for chip_address in chip_addresses[:]:
    i2c_conn.calibratePLL(chip_address, chip=None)

# Disconnect I2C Device

## DO THIS IF YOU PLAN TO DO NOISE STUDIES

In [ ]:
del i2c_conn